# What happens:
- Makes the selection and prepares the email
- Then you just make the screenshot and add the contact per Unit 

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import win32com.client as win32

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/RBUK_Data"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0194"

### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Save selection
### Also see how much time it took

In [4]:
%%time
d = pd.read_sql_query("""select distinct concat(sp.book_code,'_SEC') as sec, s.security_reference, s.issuer_code, sp.position_initial_value, nominal
from security s
inner join security_positions sp
on s.security_reference = sp.security_reference
where s.issuer_code like '%DUMMY%' and s.security_family = 'B' and sp.position_initial_value >0""", connection_RAY)

CPU times: total: 15.6 ms
Wall time: 338 ms


### Show selection

In [8]:
d
#[d.sec.str.contains('RBHR')]

,sec,security_reference,issuer_code,position_initial_value,nominal
0,RBHR_B_SEC,HRZGHOO237A3,*DUMMY_UNRATED*,3.450562e+05,3.450562e+05
1,RBHR_B_SEC,HRZGHOO237A3,*DUMMY_UNRATED*,4.929375e+05,4.929375e+05
2,RBHR_B_SEC,HRZGHOO237A3,*DUMMY_UNRATED*,5.881829e+05,5.881829e+05
3,RBHR_T_SEC,HRZGHOO237A3,*DUMMY_UNRATED*,4.929375e+06,4.929375e+06
4,RBHR_B_SEC,HRZGHOO237A3,*DUMMY_UNRATED*,4.409496e+06,4.409496e+06
5,RBHR_B_SEC,HRZGHOO237A3,*DUMMY_UNRATED*,4.218682e+06,4.218682e+06
6,RBHR_B_SEC,HRZGHOO237A3,*DUMMY_UNRATED*,1.971750e+06,1.971750e+06
7,RBHR_T_SEC,HRZGHOO237A3,*DUMMY_UNRATED*,1.478812e+05,1.478813e+05
8,RBHR_B_SEC,HRZGHOO237A3,*DUMMY_UNRATED*,1.183050e+06,1.183050e+06
9,HO_T_SEC,GB00B15KY765,*DUMMY_UNRATED*,2.275698e+04,2.271858e+04


### Send the email 

In [7]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = ''
mail.Cc = 'flori.luci@rbinternational.com;'
mail.Subject = ' - Securities with issuer code DUMMY (' + rd + ')'
mail.Body = '''Dear colleagues,

Please your support to update/correct following *Dummy* issuer codes in the SECURITIY table:

Thanks in advance!

KR,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''


mail.Display()